In [1]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode','longitude', 'latitude',  'geometry'])

In [5]:
gdf_width = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/道路/2021/RdCL/3.shp', encoding='shift-jis') #このデータは道路中心線

In [6]:
#座標系を変更
src_proj = 6668 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_width.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_width = gdf_width.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [7]:
#高速道路のデータフレームの頭にmainroadをつける

#すべてのカラムを取得
mainroad_columns = gdf_width.columns.to_list()

#ループで各カラムを取得
for i in mainroad_columns[:22]:
  new_column = 'mainroad_' + i
  gdf_width.rename(columns={f'{i}': f'{new_column}'}, inplace=True)

In [8]:
#重心座標のPointを100mメッシュのデータフレームに追加
gdf_osyu_100m_mesh['center_geometry'] = [Point(x) for x in zip(gdf_osyu_100m_mesh['longitude'], gdf_osyu_100m_mesh['latitude'])]

In [9]:
# cKDTreeを使用して最近傍と距離を求める
line_coords = gdf_width.geometry.apply(lambda geom: list(geom.coords[0]))
point_coords = gdf_osyu_100m_mesh.center_geometry.apply(lambda geom: list(geom.coords[0]))

line_tree = cKDTree(line_coords.tolist())
point_tree = cKDTree(point_coords.tolist())

# 最近傍の検索
distances, line_indices = line_tree.query(point_coords.tolist(), k=1)
nearest_lines = gdf_width.iloc[line_indices]

# 距離を結果に追加
nearest_lines['distance_mainroad'] = distances

# 結果を表示
print(nearest_lines)

#lineのgeometryxはいらないので削除
nearest_lines.drop('geometry', axis=1, inplace=True)

#100mメッシュに対して同じ高速道路が最近傍だとindexが複数できてしまうので並び替え
nearest_lines.reset_index(drop=True, inplace=True)

#100mメッシュのデータフレームに最近棒の高速道路を追加
gdf_width = gdf_osyu_100m_mesh.join(nearest_lines)

            mainroad_rID mainroad_lfSpanFr  mainroad_lfSpanTo  \
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
...                  ...               ...                ...   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   
584914  50700-14400-i-19        2020-09-30                NaN   

        mainroad_tmpFlg mainroad_orgGILvl mainroad_ftCode mainroad_admCode  \
584914                0             25000            2701              NaN   
584914                0             25000            2701      

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-9-4a55ffebac72>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearest_lines.drop('geometry', axis=1, inplace=True)


In [10]:
#座標系を変更
src_proj = 6678 # 変換前の座標系を指定
dst_proj = 4326  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_width.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_width = gdf_width.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [11]:
# #geometryが二つあると出力できないのでcenter_geometryを削除
gdf_width.drop(columns=['longitude', 'latitude', 'geometry', 'center_geometry'], axis=1, inplace=True)

In [12]:
gdf_width.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市 最近傍道路/奥州市_主要道路.csv', encoding='shift-jis')